In [1]:
from kachaka_base import *
import pickle
import time

In [2]:
class VirtualKachaka(KachakaBase):
    def __init__(self, data_num):
        self.path = "data/sensor_data" + data_num + "/"
        self.update_sensor_data()

    def move_to_pose(self, distination: Pose):
        pass

    def is_moving_finished(self):
        pass

    def update_sensor_data(self):
        ratio = 1000
        with open(self.path + "position.pkl", "rb") as file:
            position = pickle.load(file)

        self.pose = Pose(
            position["x"] * ratio, position["y"] * ratio, position["theta"]
        )

        with open(self.path + "theta.pkl", "rb") as file:
            theta = pickle.load(file)

        with open(self.path + "dist.pkl", "rb") as file:
            dist = pickle.load(file)

        self.lidar_data = LidarData(dist, theta, self.pose)


In [3]:
%matplotlib
# %matplotlib ipympl

# 初期化 -------------------------------------------------------------
initial_box_pose = Pose(1000, 1000, 0)
red_box_goal = Pose(1700, -500, 0)
blue_box_goal = Pose(250, 1500, 0)
box = Box(initial_box_pose)
kachaka = VirtualKachaka("5")
map = GridMap(
    Size(2340, 2890),
    grid_size=Size(150, 150),
    origin_offset=Pose(400, 950, 0),
    start=Pose(-200, 0, 0),
    initial_box_pose=initial_box_pose,
    red_box_goal=red_box_goal,
    blue_box_goal=blue_box_goal,
)
logger = TextLogger()
path_planner = StraightPathPlanner()
controller = Controller(kachaka, box, map, path_planner, logger)

(x_lim, y_lim) = map.get_axes_lim()
# 表示マージン確保
margin = 1000
x_lim = (x_lim[0] - margin, x_lim[1] + margin)
y_lim = (y_lim[0] - margin, y_lim[1] + margin)
plotter = Plotter(x_lim, y_lim)

# メインループ ---------------------------------------------------------
while True:
    controller.update()
    plotter.update(map, box, kachaka, controller.path)
    time.sleep(0.1)

Using matplotlib backend: <object object at 0x0000019597077550>
センサ情報を更新します
箱の前まで移動します
箱の色を読みとります
青の箱を検出しました
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します
箱を運ぶ経路を生成します


KeyboardInterrupt: 

: 